# 0. Install and Import Dependencies

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import csv
import warnings
import tensorflow as tf
import pandas as pd
import pickle
warnings.filterwarnings('ignore')

In [2]:
mp_drawing = mp.solutions.drawing_utils #Drawing helpers
mp_pose = mp.solutions.pose

# 1. Load Model

In [3]:
# path to the model
dir_model = "../model/xyz/"

# you can try different model
# dir_model = "../Model/xyz_hand/"
# dir_model = "../Model/xy/"
# dir_model = "../Model/xy_hand/"

loaded_model = tf.saved_model.load(dir_model)
inferer = loaded_model.signatures["serving_default"]

# 2. Setup OpenCV and MediaPipe to Predict

In [12]:
dir_video_test = ('../../data/momentum/test4.mp4')
cap = cv2.VideoCapture(dir_video_test)
current_stage = ''

#initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    # stream the video
    while cap.isOpened():
        ret, frame = cap.read()
        
        # recolor feed
        # image = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # make detection
        results = pose.process(image)
        
        # recolor image back to BGR for rendering
        image.flags.writeable = False
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        # try to predict
        try:
            # model xyz
            row = np.array([[res.x, res.y, res.z] for res in results.pose_world_landmarks.landmark]).flatten()
            X = pd.DataFrame([row])
            momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # model xyz_hand
            # row = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark[13:22]]).flatten()
            # X = pd.DataFrame([row])
            # momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            # momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # model xy
            # row = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark[]]).flatten()
            # X = pd.DataFrame([row])
            # momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            # momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # model xy_hand
            # row = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark[13:22]]).flatten()
            # X = pd.DataFrame([row])
            # momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            # momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            print(momentum_class, momentum_prob)

            if momentum_class == 0 and momentum_prob >= 0.9 :
                current_stage = 'Nice!'
            elif momentum_class == 1 and momentum_prob >= 0.9 :
                current_stage = 'Use momentum!'

            # get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

            #Display class
            cv2.putText(image, 'Class', (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, current_stage, (95, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

            #Display Prob
            cv2.putText(image, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(momentum_prob, 2)), (15, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
            
        except Exception as e:
            print(e)
            
        # Stream vid resultq
        cv2.imshow("Raw Cam Feed", image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1 0.9999328
1 0.9998091
1 0.99992275
1 0.9996419
1 0.9987937
1 0.9854994
1 0.9390215
1 0.8694836
1 0.89359367
1 0.88157797
1 0.8679315
1 0.8858536
1 0.9239015
1 0.9803217
1 0.9897637
1 0.99429816
1 0.9987986
1 0.9996508
1 0.9999336
1 0.9999427
1 0.99995553
1 0.9999865
1 0.9999627
1 0.9999903
1 0.9999995
1 0.99999905
1 0.9999972
1 0.9999994
1 0.9999989
1 0.9999991
1 0.9999995
1 0.99999845
1 0.9999975
1 0.99999684
1 0.9999947
1 0.9999904
1 0.99998784
1 0.99998516
1 0.99995756
1 0.99984914
1 0.99990416
1 0.9997973
1 0.9997832
1 0.99993664
1 0.9996221
1 0.9998342
1 0.99975467
1 0.9995322
1 0.99939114
1 0.9992619
1 0.9993446
1 0.99980944
1 0.99971867
1 0.9996913
1 0.9996961
1 0.99964845
1 0.9998235
1 0.99952596
1 0.99999154
1 0.99999464
1 0.9999917
1 0.9999885
1 0.99998844
1 0.9999885
1 0.999988
1 0.9999872
1 0.999988
1 0.99998564
1 0.9999694
1 0.9999728
1 0.9999462
1 0.9999515
1 0.9999013
1 0.99967813
1 0.9996078
1 0.9996618
1 0.99940354
1 0.9997041
1 0.9985302
1 0.99780214
1 0.9940073
1 0